In [57]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost/taqwa_corp?charset=utf8mb4")

dfParts = pd.read_sql("SELECT * FROM nparts ORDER BY PartNo ", engine)

dfParts = dfParts.reset_index(drop=True)  # make sure indexes pair with number of rows

#print(dfParts) # PartNo, INV

sql = """SELECT invoices.PartNo, COUNT(invoices.PartNo) as TTL 
FROM invoices 
JOIN orders ON orders.OID = invoices.OID 
WHERE orders.oDate <= '2024-09-29'
GROUP BY invoices.PartNo 
ORDER BY invoices.PartNo"""

dfUpdate = pd.read_sql(sql, engine)

dfUpdate = dfUpdate.reset_index(drop=True)  # make sure indexes pair with number of rows

#print(dfUpdate)

#using for iteration and index, updating each value
for i in dfParts.index:
    dfParts.at[i,'Inv'] = dfParts.at[i,'Inv'] -  dfUpdate.at[i,'TTL']

#print(dfParts) # PartNo, INV

#considered joining or merging cols and the subtracting and dropping col, but the code aboe was more efficent IMO    

dfParts.to_sql("nparts", engine, if_exists="replace", index=False)



62